In [2]:
import numpy as np
import matplotlib.pyplot as plt
import autokeras as ak
import tensorflow as tf
import os

In [3]:
import logging

logging.basicConfig(level = logging.INFO)

gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#   # Restrict TensorFlow to only use the first GPU
try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    tf.config.experimental.set_virtual_device_configuration(
    gpus[0],
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5000)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
#     logging.info(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
except RuntimeError as e:
# Visible devices must be set before GPUs have been initialized
    logging.info(e)

In [3]:
exposure = 'p25'
data = np.load('/home/ML4NO/Data/n1000000{}_0910_classification.npz'.format(exposure))

In [4]:
data_selection = 1 # 0 for all, 1 for lowE(<5GeV), 2 for high(>5GeV)

if data_selection == 0:
    data_all = np.column_stack([data['ve_dune'], data['vu_dune'], data['vebar_dune'], data['vubar_dune']])
    directory = './models_all/'
elif data_selection == 1:
    data_all = np.column_stack([data['ve_dune'][:,:36], data['vu_dune'][:,:36], data['vebar_dune'][:,:36], data['vubar_dune'][:,:36]])
    directory = './models_lowE/'
elif data_selection == 2:
    data_all = np.column_stack([data['ve_dune'][:,36:], data['vu_dune'][:,36:], data['vebar_dune'][:,36:], data['vubar_dune'][:,36:]])
    directory = './models_highE/'
target = data['cpv']

x_train = data_all[:10000]
y_train = target[:10000]
x_train2 = data_all[10000:900000]
y_train2 = target[10000:900000]
x_test = data_all[900000:]
y_test = target[900000:]

In [5]:
clf = ak.StructuredDataClassifier(overwrite=True, max_trials=30)
clf.fit(x_train, y_train,
           validation_split = 0.1,
           batch_size=64,
           epochs=20,
           verbose=1,
           shuffle = True
        )

Trial 30 Complete [00h 00m 44s]
val_accuracy: 0.9901315569877625

Best val_accuracy So Far: 0.9912280440330505
Total elapsed time: 00h 22m 00s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Epoch 1/20
157/157 [==============================] - 3s 12ms/step - loss: 0.5826 - accuracy: 0.6904
Epoch 2/20
157/157 [==============================] - 2s 13ms/step - loss: 0.1890 - accuracy: 0.9393
Epoch 3/20
157/157 [==============================] - 2s 12ms/step - loss: 0.1286 - accuracy: 0.9615
Epoch 4/20
157/157 [==============================] - 2s 12ms/step - loss: 0.1057 - accuracy: 0.9697
Epoch 5/20
157/157 [==============================] - 2s 12ms/step - loss: 0.0931 - accuracy: 0.9737
Epoch 6/20
157/157 [==============================] - 2s 12ms/step - loss: 0.0846 - accuracy: 0.9757
Epoch 7/20
157/157 [==============================] - 2s 12ms/step - loss: 0.0799 - accuracy: 0.9796
Epoch 8/20
157/157 [==============================] - 2s 12ms/step - loss: 0.0756 - accuracy: 0.9806
Epoch 9/20
157/157 [==============================] - 2s 12ms/step - loss: 0.0726 - accuracy: 0.9804
Epoch 10/20
157/157 [==============================] - 2s 12ms/step - loss: 0.0690 - accura


FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.


INFO:tensorflow:Assets written to: ./structured_data_classifier/best_model/assets


INFO:tensorflow:Assets written to: ./structured_data_classifier/best_model/assets


In [6]:
model = clf.export_model()
model.summary()
model.evaluate(x_test, y_test)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 144)]             0         
_________________________________________________________________
multi_category_encoding (Mul (None, 144)               0         
_________________________________________________________________
normalization (Normalization (None, 144)               289       
_________________________________________________________________
dense (Dense)                (None, 32)                4640      
_________________________________________________________________
re_lu (ReLU)                 (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
re_lu_1 (ReLU)               (None, 32)                0     

[0.060965023934841156, 0.9846199750900269]

In [7]:
model.fit(x_train2, y_train2,
           validation_split = 0.1,
           batch_size=64,
           epochs=5,
           verbose=1,
           shuffle = True
         )

Epoch 1/5
12516/12516 [==============================] - 151s 12ms/step - loss: 0.0472 - accuracy: 0.9891 - val_loss: 0.0514 - val_accuracy: 0.9915
Epoch 2/5
12516/12516 [==============================] - 149s 12ms/step - loss: 0.0391 - accuracy: 0.9911 - val_loss: 0.0417 - val_accuracy: 0.9918
Epoch 3/5
12516/12516 [==============================] - 148s 12ms/step - loss: 0.0348 - accuracy: 0.9921 - val_loss: 0.0306 - val_accuracy: 0.9938
Epoch 4/5
12516/12516 [==============================] - 152s 12ms/step - loss: 0.0322 - accuracy: 0.9929 - val_loss: 0.0280 - val_accuracy: 0.9939
Epoch 5/5
12516/12516 [==============================] - 149s 12ms/step - loss: 0.0307 - accuracy: 0.9932 - val_loss: 0.0302 - val_accuracy: 0.9931


In [8]:
model_index = 1
while os.path.isfile(directory + 'models/0910{}_{}.h5'.format(exposure, model_index)):
    model_index += 1
model.save(directory + 'models/0910{}_{}.h5'.format(exposure, model_index), overwrite=False)

In [9]:
furthur_index = 1
path = directory + 'models_furthurTrain/0910{}_{}_{}'
while os.path.isdir(path.format(exposure, model_index, furthur_index)):
    furthur_index += 1

In [10]:
scale_steps = np.logspace(-3, 0, 30)
before_train_loss = []
after_train_loss = []

for scale in scale_steps:
    x_train2_gen = np.random.normal(x_train2, np.sqrt(x_train2)*scale)
    x_test_gen = np.random.normal(x_test, np.sqrt(x_test)*scale)

    before_train_loss.append(model.evaluate(x_test_gen, y_test)[0])

    model.fit(x_train2_gen, y_train2,
               validation_split = 0.1,
               batch_size=64,
               epochs=5,
               verbose=1,
               shuffle = True
             )

    after_train_loss.append(model.evaluate(x_test_gen, y_test)[0])
    model.save(path.format(exposure, model_index, furthur_index) + '/std={}.h5'.format(scale))
model.save(path.format(exposure, model_index, furthur_index) + '.h5')

3125/3125 [==============================] - 24s 8ms/step - loss: 0.0341 - accuracy: 0.9925
Epoch 1/5
12516/12516 [==============================] - 149s 12ms/step - loss: 0.0295 - accuracy: 0.9936 - val_loss: 0.0299 - val_accuracy: 0.9928
Epoch 2/5
12516/12516 [==============================] - 150s 12ms/step - loss: 0.0291 - accuracy: 0.9937 - val_loss: 0.0221 - val_accuracy: 0.9950
Epoch 3/5
12516/12516 [==============================] - 149s 12ms/step - loss: 0.0268 - accuracy: 0.9942 - val_loss: 0.0197 - val_accuracy: 0.9959
Epoch 4/5
12516/12516 [==============================] - 147s 12ms/step - loss: 0.0267 - accuracy: 0.9943 - val_loss: 0.0230 - val_accuracy: 0.9952
Epoch 5/5
3125/3125 [==============================] - 24s 8ms/step - loss: 0.0226 - accuracy: 0.9950
Epoch 1/5
12516/12516 [==============================] - 152s 12ms/step - loss: 0.0260 - accuracy: 0.9944 - val_loss: 0.0208 - val_accuracy: 0.9955
Epoch 2/5
12516/12516 [==============================] - 150s 12ms

In [11]:
Poisson_index = 1
path = directory + 'models_PoissonTrain/0910{}_{}_{}_{}'
while os.path.isdir(path.format(exposure, model_index, furthur_index, Poisson_index)):
    Poisson_index += 1

In [12]:
x_test_gen = np.random.poisson(x_test)

for i in range(10):
    x_train2_gen = np.random.poisson(x_train2)
    
    model.fit(x_train2_gen, y_train2,
              validation_split=0.1,
               batch_size=64,
               epochs=1,
               verbose=1,
               shuffle = True
             )
    model.save(path.format(exposure, model_index, furthur_index, Poisson_index) + '/steps={}.h5'.format(i))
model.save(path.format(exposure, model_index, furthur_index, Poisson_index) + '.h5')
model.evaluate(x_test_gen, y_test)

3125/3125 [==============================] - 33s 10ms/step - loss: 0.3297 - accuracy: 0.8612


[0.3296508491039276, 0.8612099885940552]

In [13]:
pred = model.predict(x_test_gen)

In [1]:
cpc = np.where(y_test == 0)
cpv = np.where(y_test == 1)
plt.hist(pred[cpc], bins =100)
plt.hist(pred[cpv], bins = 100)
plt.show()

NameError: name 'plt' is not defined